## Problème à patch
Problème majeur, le stockage de l'écart type est fait en floatant hors l'écart entre les écarts types est inférieur à un floatant
stockage à changer sous forme d'entier plus précis

In [1]:
def lecture_csv(col:str="", limit:int=float('infinity')):
    """
    Lis le fichier loto.csv puis crée un dictionnaire de répartition de chaque boule allant de 1 à @limit
    """
    import pandas as pd;
    df = pd.read_csv("loto.csv", sep=';')


    # Creation du set de chaque numero_chance
    numero_chance:dict[int, int] = {i: 0 for i in range(1, min(10, limit)+1)}
    # Initialisation du nombre de numero_chance
    for each in df["numero_chance"]:
        if each <= limit:
            numero_chance[each] += 1

    # Creation du set de chaque boule
    boule:dict[int, int] = {i: 0 for i in range(1, min(49, limit)+1)}
    # Initialisation du nombre de boule
    for i in range(1, 5):
        # Pour les 5 sets
        for each in df[f"boule_{i}"]:
            if each <= limit:
                boule[each] += 1

    print("stat :", numero_chance if col == "chance" else boule)

    # Décision du retour
    d, size = (numero_chance, min(10, limit)) if col == "chance" else (boule, min(49, limit))
    return (d, sum(d.values()), size)

In [2]:
def moy_a_16(d:dict[int, int], t:int, size:int) -> int:

    # Formule avec Carré plutôt que Absolue
    return sum([pow(val*size - t, 2) for val in d.values()])

### Ancien ecart type

In [3]:
def chance_ecart_type_min(d:dict[int, int], t:int, size:int) -> list[tuple[int, float]]:
    # Initialisation du set de pourcentage
    percent:dict[int, int] = {}
    
    # On rajoute un seul tirage
    t += 1

    # Pour chaque numero de boule
    for i in range(1, size+1):
        # On copie le set d'origine
        data:dict[int, int] = d.copy()
        # Puis on ajoute le tirage souhaité à l'interieur
        data[i] += 1
        # On calcule le nouvel écart type et l'ajoute au pourcentage
        percent[i]:int = moy_a_16(data, t, size)
    return percent

## Nouvel écart type

In [4]:
def normal_ecart_type_min(d:dict[int, int]={}, t:int=0, size:int=0) -> list[tuple[int, float]]:
    # Initialisation du set de pourcentage
    percent = {}

    # On rajoute un seul tirage
    t += 1

    # Liste des combinaisons déjà vues
    values_already_seen:set = set()

    # Pour chaque numero de boule
    for i in range(1, size+1):
        print(i, end="\r")
        # On copie le set d'origine
        data_b1 = d.copy()
        # Puis on ajoute le tirage souhaité à l'interieur
        data_b1[i] += 1

        for j in range(1, size+1):
            if j == i:continue

            data_b2 = data_b1.copy()
            data_b2[j] += 1
            for k in range(1, size+1):
                if k == i or k == j:continue

                data_b3 = data_b2.copy()
                data_b3[k] += 1
                for l in range(1, size+1):
                    if l == k or l == j or l == i:continue

                    data_b4 = data_b3.copy()
                    data_b4[l] += 1
                    for m in range(1, size+1):
                        if m == l or m == k or m == j or m == i:continue

                        data_b5 = data_b4.copy()
                        data_b5[m] += 1

                        key:tuple = tuple(sorted((i, j, k, l, m)))

                        if key in values_already_seen:
                            continue

                        values_already_seen.add(key)

                        # On calcule le nouvel écart type et l'ajoute au pourcentage
                        percent[key]:int = moy_a_16(data_b5, t, size)
        
    return percent

In [7]:
def result(d:dict[int, int], t:int, size:int):
    # Retourne la liste triée des écarts types
    s:list
    if size == 10:
        s = sorted(chance_ecart_type_min(d, t, size).items(), key=lambda x: x[1])
    else:
        s = sorted(normal_ecart_type_min(d, t, size).items(), key=lambda x: x[1])
    
    return s

In [8]:
data, times, size = lecture_csv()
result(data, times, size)

stat : {1: 52, 2: 44, 3: 54, 4: 54, 5: 61, 6: 59, 7: 62, 8: 51, 9: 54, 10: 44, 11: 51, 12: 68, 13: 61, 14: 46, 15: 65, 16: 48, 17: 52, 18: 47, 19: 51, 20: 52, 21: 48, 22: 66, 23: 52, 24: 56, 25: 48, 26: 62, 27: 50, 28: 61, 29: 50, 30: 56, 31: 67, 32: 57, 33: 52, 34: 49, 35: 63, 36: 50, 37: 51, 38: 59, 39: 45, 40: 48, 41: 56, 42: 54, 43: 47, 44: 68, 45: 47, 46: 54, 47: 49, 48: 65, 49: 54}


[(2, 5139120),
 (10, 5139120),
 (39, 5143922),
 (14, 5148724),
 (18, 5153526),
 (43, 5153526),
 (45, 5153526),
 (16, 5158328),
 (21, 5158328),
 (25, 5158328),
 (40, 5158328),
 (34, 5163130),
 (47, 5163130),
 (27, 5167932),
 (29, 5167932),
 (36, 5167932),
 (8, 5172734),
 (11, 5172734),
 (19, 5172734),
 (37, 5172734),
 (1, 5177536),
 (17, 5177536),
 (20, 5177536),
 (23, 5177536),
 (33, 5177536),
 (3, 5187140),
 (4, 5187140),
 (9, 5187140),
 (42, 5187140),
 (46, 5187140),
 (49, 5187140),
 (24, 5196744),
 (30, 5196744),
 (41, 5196744),
 (32, 5201546),
 (6, 5211150),
 (38, 5211150),
 (5, 5220754),
 (13, 5220754),
 (28, 5220754),
 (7, 5225556),
 (26, 5225556),
 (35, 5230358),
 (15, 5239962),
 (48, 5239962),
 (22, 5244764),
 (31, 5249566),
 (12, 5254368),
 (44, 5254368)]